In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_df = pd.read_csv("training_set_VU_DM.csv")
#test_df = pd.read_csv("test_set_VU_DM.csv")

In [3]:
#df.head(10)

In [4]:
#df.isnull().sum()

In [5]:
sample_df = train_df.sample(frac=0.5, random_state=42)  # For example, 10% of the data


In [6]:
#sample_df.info()

In [7]:
def cor_map(data):    
    corr = data.corr()
    plt.figure(figsize=(12, 12))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
    plt.title('Heatmap of Correlation Between Mood, App Usage, and Other Activities')
    plt.show()

#cor_map(df)

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split

def calculate_relevance(df):
    df['relevance_score'] = 0

    # Set relevance_score to 1 where click_bool is 1 but not booked
    df.loc[(df['click_bool'] == 1) & (df['booking_bool'] == 0), 'relevance_score'] = 1

    # Set relevance_score to 5 where booking_bool is 1
    df.loc[df['booking_bool'] == 1, 'relevance_score'] = 5

    return df

# Prepare the LightGBM dataset
def prepare_lgb_data(features, labels, group=None):
    dataset = lgb.Dataset(data=features, label=labels, group=group, free_raw_data=False)
    return dataset

# Training the LightGBM model
def train_model(train_data, params):
    model = lgb.train(params, train_data, num_boost_round=100)
    return model

def get_time_of_day(hour):
    if 5 <= hour < 12:
        return 1
    elif 12 <= hour < 17:
        return 2
    elif 17 <= hour < 21:
        return 3
    else:
        return 4
    
def get_season(month):
    if month in (12, 1, 2):
        return 'Winter'
    elif month in (3, 4, 5):
        return 'Spring'
    elif month in (6, 7, 8):
        return 'Summer'
    elif month in (9, 10, 11):
        return 'Fall'

def get_seasons(month):
    if month in (12, 1, 2):
        return 1
        #return 'Winter'
    elif month in (3, 4, 5):
        return 2
        #return 'Spring'
    elif month in (6, 7, 8):
        return 3
        #return 'Summer'
    elif month in (9, 10, 11):
        return 4
       # return 'Fall'
        
def feature_engineering(df):
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['time_of_day'] = df['date_time'].dt.hour.apply(get_time_of_day)
    df['is_weekend'] = df['date_time'].dt.dayofweek >= 5
    df['month'] = df['date_time'].dt.month
    df['day'] = df['date_time'].dt.day
    #df['month'] = df['date_time'].dt.month
    #df = pd.get_dummies(df, columns=['month'], prefix='month')
    #df = pd.get_dummies(df, columns=['month'])
   # df['rank_season'] = df['date_time'].dt.month.apply(get_season)
    #df = pd.get_dummies(df, columns=['rank_season'])
    df['season'] = df['date_time'].dt.month.apply(get_seasons)  
    df['srch_count'] = df['srch_adults_count'] + df['srch_adults_count']
    df['price_per_night'] = df['price_usd'] / df['srch_length_of_stay']
    df['price_per_person'] = df['price_usd'] /df['srch_count']
    #df['star_rating_difference'] = df['prop_starrating'] - df['visitor_hist_starrating'].fillna(df['prop_starrating'])
    df['is_domestic'] = (df['prop_country_id'] == df['visitor_location_country_id']).astype(int)
    df['has_hist_starrating'] = df['visitor_hist_starrating'].notna().astype(int)
    df['has_hist_adr_usd'] = df['visitor_hist_adr_usd'].notna().astype(int)
    df['prop_location_overall'] = df['prop_location_score1'] + df['prop_location_score2']
    df['prop_location_mean'] = (df['prop_location_score1'] + df['prop_location_score2'])/2
    
    ##comp
    competitors = range(1, 9)  
    # average comparisons/diff
    df['avg_rate_comparison'] = df[[f'comp{i}_rate' for i in competitors]].mean(axis=1, skipna=True)
    df['avg_inventory_comparison'] = df[[f'comp{i}_inv' for i in competitors]].mean(axis=1, skipna=True)
    df['avg_rate_percent_diff'] = df[[f'comp{i}_rate_percent_diff' for i in competitors]].mean(axis=1, skipna=True)
    
    df['count_rate_positive'] = df[[f'comp{i}_rate' for i in competitors]].apply(lambda x: (x == 1).sum(), axis=1)
    df['count_rate_negative'] = df[[f'comp{i}_rate' for i in competitors]].apply(lambda x: (x == -1).sum(), axis=1)
    df['count_rate_neutral'] = df[[f'comp{i}_rate' for i in competitors]].apply(lambda x: (x == 0).sum(), axis=1)
    
   # for i in competitors:
    #    df[f'comp{i}_competitive_score'] = df[f'comp{i}_rate'].fillna(0) + df[f'comp{i}_inv'].fillna(0)
    #df['total_competitive_score'] = df[[f'comp{i}_competitive_score' for i in competitors]].sum(axis=1)
    
    df['percent_no_inv'] = df[[f'comp{i}_inv' for i in competitors]].apply(lambda x: (x == 1).mean(), axis=1)

    df['max_price_diff_percent'] = df[[f'comp{i}_rate_percent_diff' for i in competitors]].max(axis=1, skipna=True)
    df['min_price_diff_percent'] = df[[f'comp{i}_rate_percent_diff' for i in competitors]].min(axis=1, skipna=True)
    
    return df

def plot_feature_importances(importances, feature_names):
    # Sort features according to importance
    sorted_idx = np.argsort(importances)
    sorted_importances = importances[sorted_idx]
    sorted_features = [feature_names[i] for i in sorted_idx]
    
    # Create the plot
    plt.figure(figsize=(20, 15))
    plt.barh(sorted_features, sorted_importances, color='blue')
    plt.xlabel('Importance')
    plt.title('Feature Importance')
    plt.show()

def main():
    df = calculate_relevance(sample_df)
    df = feature_engineering(sample_df)
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    
    train_labels = train_df['relevance_score']
    train_features = train_df.drop(['srch_id', 'prop_id', 'click_bool', 'booking_bool', 'position', 'relevance_score', 'date_time', 'gross_bookings_usd'], axis=1)
    group = train_df.groupby('srch_id').size().tolist()

    if sum(group) == len(train_features):
        train_data = prepare_lgb_data(train_features, train_labels, group)
        params = {
            'objective': 'lambdarank',
            'learning_rate': 0.1,
            'num_leaves': 70,
            'metric': 'ndcg',
            'ndcg_eval_at': [5],
            'verbose': 1
        }
        model = lgb.train(params, train_data, num_boost_round=100)
        feature_importances = pd.DataFrame({
            'Feature': train_features.columns.tolist(),
            'Importance': model.feature_importance(importance_type='gain')
        }).sort_values(by='Importance', ascending=False)
        
        print(feature_importances)
        plot_feature_importances(model.feature_importance(), train_features.columns.tolist())
    else:
        print(f"Error: Sum of group counts ({sum(group)}) does not match the number of training features ({len(train_features)})")

if __name__ == '__main__':
    main()

In [40]:
sample_df['comp2_inv'].nunique()

3

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
#def load_data(filename):
 #   return pd.read_csv(filename)

# Calculate the relevance scores for training data
def calculate_relevance(df):
    df['relevance_score'] = 0

    # Set relevance_score to 1 where click_bool is 1 but not booked
    df.loc[(df['click_bool'] == 1) & (df['booking_bool'] == 0), 'relevance_score'] = 1

    # Set relevance_score to 5 where booking_bool is 1
    df.loc[df['booking_bool'] == 1, 'relevance_score'] = 5

    return df

# Prepare the Dataset for LightGBM
def prepare_lgb_data(features, labels=None, group=None):
    if labels is not None and group is not None:
        dataset = lgb.Dataset(data=features, label=labels, group=group, free_raw_data=False)
    else:
        dataset = lgb.Dataset(data=features, free_raw_data=False)
    return dataset

# Train the LightGBM model
def train_model(train_data, params):
    model = lgb.train(params, train_data, num_boost_round=100)
    return model

# Predict using the LightGBM model
def predict(model, data):
    predictions = model.predict(data)
    return predictions

def get_time_of_day(hour):
    if 5 <= hour < 12:
        return 1
    elif 12 <= hour < 17:
        return 2
    elif 17 <= hour < 21:
        return 3
    else:
        return 4

def cleaning_data(df):
    df.drop(['visitor_hist_starrating', 'visitor_hist_adr_usd'], axis=1, inplace=True)
    mean_scores = df.groupby('prop_id')['prop_review_score'].mean()
    df['prop_review_score'] = df.groupby('prop_id')['prop_review_score'].transform(lambda x: x.fillna(x.mean()))
    


    
def feature_engineering(df):
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['time_of_day'] = df['date_time'].dt.hour.apply(get_time_of_day)
    df['is_weekend'] = df['date_time'].dt.dayofweek >= 5
    df['price_per_night'] = df['price_usd'] / df['srch_length_of_stay']
    #df['star_rating_difference'] = df['prop_starrating'] - df['visitor_hist_starrating'].fillna(df['prop_starrating'])
    df['is_domestic'] = (df['prop_country_id'] == df['visitor_location_country_id']).astype(int)
    df['has_hist_starrating'] = df['visitor_hist_starrating'].notna().astype(int)
    df['has_hist_adr_usd'] = df['visitor_hist_adr_usd'].notna().astype(int)
    df['prop_location_overall'] = (df['prop_location_score1'] + df['prop_location_score2'])/2
    df['srch_count'] = df['srch_adults_count'] + df['srch_adults_count']
    
    return df

# Main function to run the program
def main():
    train_df = feature_engineering(train_df)
    test_df = feature_engineering(test_df)
    
    train_df = calculate_relevance(train_df)
    train_data = prepare_lgb_data(train_df.drop(['srch_id', 'prop_id', 'click_bool', 'booking_bool', 'position', 'relevance_score', 'date_time', 'gross_bookings_usd'], axis=1),
                                  train_df['relevance_score'], train_df.groupby('srch_id').size().tolist())
    test_data = prepare_lgb_data(test_df.drop(['srch_id', 'prop_id', 'date_time'], axis=1))

    params = {
        'objective': 'lambdarank',
        'learning_rate': 0.1,
        'num_leaves': 70,
        'metric': 'ndcg',
        'ndcg_eval_at': [5]
    }

    model = train_model(train_data, params)
    predictions = predict(model, test_data.data)

    test_df['predicted_relevance'] = predictions
    ordered_results = test_df.sort_values(by=['srch_id', 'predicted_relevance'], ascending=[True, False])
    final_output = ordered_results[['srch_id', 'prop_id']]

    # Save the final output to a CSV file
    final_output.to_csv('sorted_properties_by_relevance.csv', index=False)
    print("Model training and prediction complete. Results saved to sorted_properties_by_relevance.csv.")
    return final_output


final_output = main()
print(final_output.head())

In [ ]:
#df.info()

# Features
1. *visitor_hist_starrating* = The mean star rating of hotels the customer has previously purchased
    - 4706481
    - null signifies there is no purchase history on the customer
    - create if history or not
    - impute but too much missing?
    
2. *visitor_hist_adr_usd* = The mean price per night of the hotels the customer has previously purchased
    - 4705359
    - null signifies there is no purchase history on the customer
    - check if purchase history consistent between the 2
3. *prop_location_score2* = A (second) score outlining the desirability of a hotel’s location:
    - just use score1 if score2 doesn't have a big influence, if not imputation
    - merge with 1, create combined value
4. *prop_review_score* = a 0 means there have been no reviews, null that the information is not available
    - mean customer review score
    - also check with the review_star, if too similar ditch one?
5. *srch_query_affinity_score* A null signifies there are no data (i.e. hotel did not register in any searches) 
    - make a separate variable if hotel had any searches or no
    - 4640941 a lot of the data missing, maybe remove column?
6. *orig_destination_distance*  Physical distance between the hotel and the customer at the time of search. 
    - A null means the distance could not be calculated
    - ignore
7. *comp*:
    - null signifies there is no competitive data
    - check which 1 of the 3 more relevant
8. *total number of occupants*